In [1]:
import flywheel
import datetime

# Create client
fw = flywheel.Client()

# define containers & files
gear = fw.lookup('gears/xcpengine-fw')
project = fw.lookup('davis/presurgicalEpilepsy')
design_file = project.get_file('cbf_WT.dsn')

# set date for label
x = datetime.datetime.now()
date_str = '%s/%s/%s %s:%s' % (x.month, x.day, x.year, x.hour, x.minute)

# set inputs and config for gear
#inputs = {'antsct': , 'designfile': design_file, 'img':, 'm0':}
#config = {'analysis_type': cbf}



In [21]:
# run gear
session = fw.lookup('davis/presurgicalEpilepsy/P61/ses-01')
for a in session.analyses:
   print(a.label)

SB_fmriprep 8-13-2019 15:21
xcpengine-fw 08/22/2019 10:12:16 STRUC MINIMAL WT
xcpengine-fw 08/22/2019 14:40:49 STRUC MINIMAL WT
xcpengine-fw 08/26/2019 11:41:37 CBF WT
xcpengine-fw 08/26/2019 13:18:18 CBF WT
xcpengine-fw 08/26/2019 14:34:59 CBF WT
xcpengine-fw 08/26/2019 15:02:00 CBF WT


In [ ]:
# run gear
for ses in project.sessions():
    
    # get ASL and M0 files
    for a in ses.acquisitions():
    if 'ASL' in a.label:
        asl_container=fw.get(a.id)
        for f in asl_container.files:
            if '.nii.gz' in f.name:
                asl_img=f
    elif 'M0' in a.label:
        m0_container=fw.get(a.id)
        for f in m0_container.files:
            if '.nii.gz' in f.name:
                m0_img=f
    else:
        pass
    
    # get 
    
    #analysis_id = gear.run(analysis_label='fmriprep_test', config=config, inputs=inputs, destination=ses)